# Attention
Class = 3.

Integrate (1) Nucleus features as recored by CellProfiler Proces100
and (2) attention level recorded by the CNN.
Use Random Forest to say what CP features are predictive of attention.
Here, we do the analysis per class, just because that is easiest.

In [1]:
import datetime
import numpy as np
import pandas as pd
from RandomForestUtil import RF_Util
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [2]:
ATTFILEPATH='/home/jrm/Adjeroh/Naved/CP_80K/attention_nucleus/' # Alien
ATTFILENAME='class3_avg_temp.csv'
NUCFILEPATH='/home/jrm/Adjeroh/Naved/CP_80K/Output3/'
NUCFILENAME='Process100_Nucleus.csv'

## Load labels
The labels are the temperatures of the CNN attention heatmap,
specifically an average across the bounding box of each nucleus.

In [3]:
att_df=pd.read_csv(ATTFILEPATH+ATTFILENAME)
cols={'ImageNumber':'PatchNumber',' ObjectNumber':'ObjectNumber',' AvgTemp':'AvgTemp'}
att_df.rename(columns=cols,inplace=True)
att_df.sort_values(by=['PatchNumber','ObjectNumber'],axis=0,inplace=True)
att_df.reset_index(drop=True,inplace=True)

In [4]:
# Later, try the RandomForestRegressor. For now use bins and classification.
bins = [0, 0.3, 0.45, 0.6, 0.75, 1.0]
labels = ['to30%', 'to45%', 'to60%', 'to75%', 'to100%']
att_df['bin'] = pd.cut(att_df['AvgTemp'], bins=bins, labels=labels)
att_df

,PatchNumber,ObjectNumber,AvgTemp,bin
0,1,1,0.219112,to30%
1,1,2,0.253136,to30%
2,1,3,0.148982,to30%
3,1,4,0.280976,to30%
4,1,5,0.391533,to45%
...,...,...,...,...
101606,6364,3,0.718735,to75%
101607,6364,4,0.579273,to60%
101608,6364,5,0.199075,to30%
101609,6364,6,0.245602,to30%


In [5]:
ytrain = att_df['bin'] 
ytrain.value_counts()

to30%     22497
to100%    22086
to60%     21215
to45%     19425
to75%     16388
Name: bin, dtype: int64

## Load features
These are features of nuclei as determined by our "Process100" pipeline for CellProfiler.
These are the nuclei used to set the bounding boxes for the temperature lables.
Other than that, the features and temperatures were each derived independently.

In [6]:
nuc_df=pd.read_csv(NUCFILEPATH+NUCFILENAME)
cols={'ImageNumber':'PatchNumber'}
nuc_df.rename(columns=cols,inplace=True)
nuc_df.sort_values(by=['PatchNumber','ObjectNumber'],axis=0,inplace=True)
nuc_df.reset_index(drop=True,inplace=True)

In [7]:
Xtrain = nuc_df.drop(columns=['PatchNumber','ObjectNumber'])
Xtrain

,AreaShape_Area,AreaShape_BoundingBoxArea,AreaShape_BoundingBoxMaximum_X,AreaShape_BoundingBoxMaximum_Y,AreaShape_BoundingBoxMinimum_X,AreaShape_BoundingBoxMinimum_Y,AreaShape_Center_X,AreaShape_Center_Y,AreaShape_CentralMoment_0_0,AreaShape_CentralMoment_0_1,...,Texture_Variance_Hematoxylin_4_02_256,Texture_Variance_Hematoxylin_4_03_256,Texture_Variance_Hematoxylin_5_00_256,Texture_Variance_Hematoxylin_5_01_256,Texture_Variance_Hematoxylin_5_02_256,Texture_Variance_Hematoxylin_5_03_256,Texture_Variance_Hematoxylin_7_00_256,Texture_Variance_Hematoxylin_7_01_256,Texture_Variance_Hematoxylin_7_02_256,Texture_Variance_Hematoxylin_7_03_256
0,223,330,293,15,271,0,280.201794,5.852018,223.0,-1.829648e-13,...,585.532136,400.390101,559.863956,784.976400,641.261347,402.097750,613.053883,1015.538889,820.496571,391.738196
1,725,1000,218,25,178,0,197.977931,10.463448,725.0,9.237056e-14,...,497.718433,545.823261,558.218484,562.826135,504.393591,570.679814,600.901961,572.911912,539.498863,657.375035
2,411,588,52,28,31,0,40.296837,12.133820,411.0,-3.588241e-13,...,507.285994,417.003835,559.389690,612.057261,527.195474,413.443783,603.223242,764.249463,573.008173,436.770501
3,426,594,62,73,44,40,51.859155,55.612676,426.0,1.989520e-13,...,427.497076,480.299294,491.016816,456.018600,426.776905,508.279184,566.183067,518.998271,420.803973,593.294902
4,493,656,147,90,131,49,138.935091,70.255578,493.0,3.375078e-14,...,519.715947,498.768060,554.062740,496.513905,527.690620,536.811380,624.890491,423.286477,531.450178,634.128941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101606,265,336,270,83,254,62,261.615094,71.943396,265.0,8.260059e-14,...,785.133809,785.319444,808.125625,774.810684,772.121958,794.879013,922.240000,964.554784,671.690461,824.172808
101607,434,552,246,96,223,72,234.993088,83.635945,434.0,-5.684342e-14,...,1117.651714,1138.161084,1118.242525,1039.801483,1137.780751,1224.910835,1180.279948,1077.623979,1195.047867,1352.495922
101608,223,270,92,165,74,150,82.834081,157.390135,223.0,2.131628e-13,...,794.340117,762.984256,790.941174,759.607455,804.644581,775.692471,779.676115,718.209104,817.973600,713.814082
101609,212,483,109,181,88,158,97.764151,166.754717,212.0,1.705303e-13,...,1895.464517,1657.754137,1941.077583,1758.442817,1882.411074,1662.459483,2056.405675,385.187500,1611.493220,1653.214587


In [8]:
att_df=None
nuc_df=None
print('Count Nan before:',Xtrain.isna().sum().sum())
Xtrain.fillna(0,inplace=True)
print('Count Nan after:',Xtrain.isna().sum().sum())

Count Nan before: 305043
Count Nan after: 0


## Measure agreement (with a random forest classifier)
Can the RF predict the temperature of each nucleus given only the CellProfiler features?

In [9]:
print(datetime.datetime.now())
print("Shuffle...")
Xtrain,ytrain=shuffle(Xtrain,ytrain)  #set random_state for reproducibility

2022-06-09 12:05:44.319543
Shuffle...


In [10]:
print(datetime.datetime.now())
print('Cross validation...')
rf1 = RF_Util()
rf1.set_train(Xtrain,ytrain)
cv_scores = rf1.cross_validation()
print(cv_scores)
print('Accuracy mean %.4f +/- %.4f' % (cv_scores.mean(),cv_scores.std()))

2022-06-09 12:05:44.535817
Cross validation...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time= 2.7min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.7min remaining:    0.0s


[CV] END .................................................... total time= 2.6min
[CV] END .................................................... total time= 2.5min
[CV] END .................................................... total time= 2.6min
[CV] END .................................................... total time= 2.6min
[0.92948876 0.9244661  0.9238756  0.92564708 0.92456451]
Accuracy mean 0.9256 +/- 0.0020


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 13.0min finished


## Measure feature importance

In [11]:
print(datetime.datetime.now())
print('Re-train on full training set...')
rf1 = None
rf2 = RF_Util()
rf2.set_train(Xtrain,ytrain)
rf2.fit()
print(datetime.datetime.now())
print('...and rank the features by importance.')
top = rf2.important_features()
pd.set_option('display.max_rows', None)
top.loc[:200]

2022-06-09 12:18:44.707925
Re-train on full training set...
2022-06-09 12:22:01.653020
...and rank the features by importance.


,0,1
0,0.071229,Location_CenterMassIntensity_X_Hematoxylin
1,0.070382,Location_Center_X
2,0.063901,AreaShape_Center_X
3,0.063776,AreaShape_BoundingBoxMaximum_X
4,0.062157,AreaShape_BoundingBoxMinimum_X
5,0.051451,Location_MaxIntensity_X_Hematoxylin
6,0.039921,Location_Center_Y
7,0.037429,Location_CenterMassIntensity_Y_Hematoxylin
8,0.037384,AreaShape_Center_Y
9,0.036304,AreaShape_BoundingBoxMaximum_Y
